## 실무에 적용하기에는 데이터 양이나 학습양이 적은 듯 함
1. KoGPT-2는 상대적으로 모델의 크기가 작습니다. 
2. 챗봇의 성능을 올리고 싶다면 데이터의 양을 증량하거나 T5-Large와 같은 보다 큰 모델을 시도하시기를 권합니다.

transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT와 마찬가지로 GPT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [4]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
import pandas as pd
import tqdm
import urllib.request


TFGPT2LMHeadModel.from_pretrained('GPT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 GPT 구조를 로드합니다. AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다. 토크나이저를 로드할 때 텍스트 생성을 시작할 때 사용하는 시작 토큰을 bos_token, 텍스트 생성을 종료할 때 사용하는 종료 토큰을 eos_token이라는 이름으로 지정할 수 있습니다. 여기서는 </s>라는 토큰을 시작 토큰이자 종료 토큰으로 사용하고자 합니다.

In [2]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

GPT에는 특별한 용도로 사용하기 위해서 할당해놓은 특별 토큰(special token)들이 존재합니다. Ko-GPT2에서 사용되는 특별 토큰들의 정수 그리고 반대로 정수 1, 2, 3, 4는 각각 어떤 토큰들을 의미하는지 확인해봅시다. <usr> 토큰은 보통 사용자. 즉 사람의 질문 내용 <sys>는 시스템을 의미하는데 쉽게 얘기하면 사람의 질문에 대한 시스템 답변을 의미함

In [3]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))


1
1
3
----------
</s>
<usr>
<pad>
<sys>


주로 문장의 시작을 의미하는 용도로 사용되는 BOS 토큰과 주로 문장의 종료를 의미하는 용도로 사용되는 EOS token이 정수로는 몇 번인지 확인하기 위해서 tokenizer.bos_token_id와 tokenizer.eos_token_id를 각각 출력했습니다. 또한 패딩 토큰이 정수로는 몇 번인지 확인하기 위해서 tokenizer.pad_token_id를 출력했습니다. 확인 결과, KoGPT-2에서는 BOS 토큰과 EOS 토큰이 동일한 정수 1로 맵핑되는 것으로 확인되었으며 (이것은 KoGPT-2 제작자들이 이렇게 정한 것입니다.) 패딩 토큰은 정수 3으로 맵핑되는 것을 확인했습니다.

반대로 정수 1, 2, 3, 4를 단어로 표현할 경우를 출력해보았습니다. 정수 1. 즉, BOS 토큰이기도 하고, EOS 토큰이기도 한 토큰은 </s>로, 패딩 토큰인 정수 3은 <pad>로 출력되는 것을 확인했습니다. 정수 2와 정수 4의 경우에는 <usr>와 <sys>라는 토큰으로 맵핑되어져 있었는데, 이는 GPT로 대화 모델을 만들 경우에 각각 유저(User)와 시스템(System)을 구분하기 위한 용도입니다. 이번 실습에서도 실제로 이 두 토큰을 이용하여 챗봇을 구현할 것이므로 어떠한 의미인지는 뒤에서 실습을 통해 이해해보겠습니다.

### 챗봇 데이터 로드

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
print('챗봇 데이터의 개수 :', len(train_data))

챗봇 데이터의 개수 : 11823


In [6]:
train_data[:5]

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [7]:
# 챗봇 데이터를 전처리하기 위한 함수 구현
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
    yield bos_token + sent + eos_token


위의 전처리는 <usr> 다음에 사용자의 질문을 부착하고, 그 후 <sys> 다음에 챗봇의 답변을 부착하는 형식으로 전처리를 진행합니다. 그리고 앞 뒤에 시작 토큰(BOS 토큰)과 종료 토큰(EOS 토큰)을 부착하는 전처리입니다. 앞에서 확인했듯이 두 토큰은 동일하며 </s>입니다. 전처리 후에 데이터가 어떻게 변경되는지는 뒤에서 확인합니다. 배치 크기를 32로 하고, 패딩 토큰으로 패딩을 진행합니다. 위에서 확인했듯이 패딩 토큰의 정수는 3입니다.

In [8]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

# 배치 크기만큼 데이터를 구성하되 패딩 토큰으로 패딩을 진행.
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)


In [9]:
# 첫번째 데이터를 출력해보자
# 첫번째 배치 출력
for batch in dataset:
    print(batch)
    break


tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  800

데이터가 길어 데이터를 중략하였습니다. 32개의 데이터를 보면 길이는 30으로 패딩되었으며 뒤에 채워진 패딩 토큰은 숫자 3이고, 각 데이터의 맨 앞에는 정수 1, 2가 부착되어져 있습니다. 정수 1, 2는 앞에서 확인했던 것과 같이 각각 </s>와 <user>에 해당됩니다. 첫번째 배치는 현재 batch라는 변수에 저장되어져 있으므로 32개의 데이터 중 가장 첫번째 샘플을 출력해봅시다. 이때 이미 정수 인코딩이 진행된 상황이므로 tokenizer.decode()를 통해 정수 인코딩 된 결과를 다시 복원하여 출력합니다. 위의 결과에 보면 4도 포함되어 있으니 잘봐야 함. 4 이후는 시스템 답변입.

In [10]:
# 첫번째 배치(32개의 샘플) 중 첫번째 샘플 출력
print(tokenizer.decode(batch[0]))

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


위의 get_chat_data() 함수로 전처리를 진행할 당시의 의도대로 사용자의 질문 앞에 <usr>가 붙어있고, 챗봇의 답변 앞에는 <sys>가 붙어있습니다. 그리고 사용자의 질문과 챗봇의 답변 앞 뒤로는 시작 토큰과 종료 토큰에 해당하는 </s>이 붙어있으며, 그 뒤에는 패딩 토큰인 <pad>가 붙습니다. 이제 학습을 진행해봅시다.

## 챗봇 학습하기
1. 지금까지는 패딩을 한것이다. 학습을 한게 아니다.

In [11]:
# 옵티마이저 결정
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# 전체 데이터의 개수를 배치 크기로 나누면 하나의 에포크에서 실행되는 학습 횟수가 계산됨.
steps = len(train_data) // batch_size + 1

EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape: # 아래 내용을 전부다 태입에 기록한다
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_27604\1977177423.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.12708354


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.69813848


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    3] cost = 1.37551296


## 챗봇 실행하기

이제 챗봇에게 답변을 얻기 위해서 어떤 전처리를 진행해야 할 지 차근차근 정리해봅시다. 우선 앞서 모델 학습 전에 전처리를 진행했던 것처럼 사용자의 질문의 앞 뒤에 <usr>와 <sys>를 부착합니다. 그 후 해당 문장 앞에 시작 토큰에 해당하는 </s>를 부착합니다.

In [12]:
text = '오늘도 좋은 하루!'

sent = '<usr>' + text + '<sys>'

input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print('정수 인코딩 후 :', input_ids)
print('정수 인코딩을 재복원 :', tokenizer.decode(input_ids[0]))

정수 인코딩 후 : tf.Tensor([[    1     2 10070  7235 10586 12557   376     4]], shape=(1, 8), dtype=int32)
정수 인코딩을 재복원 : </s><usr> 오늘도 좋은 하루!<sys>


현재의 입력은 챗봇이 학습할 당시의 형태(</s><usr>사용자의 질문<sys>챗봇의 답변</s>)에서 챗봇의 답변이 바로 시작되기 전까지의 형태입니다. 기존에 학습하였을 당시에는 <sys> 뒤에 챗봇의 답변이 있었으므로 현재의 입력을 KoGPT-2에 입력으로 사용하면 챗봇은 학습때 봤던 데이터의 형식 그대로 챗봇의 답변을 작성하려는 관성을 갖고 있습니다. 현재의 입력을 모델의 입력으로 넣고 모델이 생성하는 문장을 얻습니다.

In [13]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence)


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


</s><usr> 오늘도 좋은 하루!<sys> 오늘도 좋은 하루네요.</s>


KoGPT-2의 출력은 사용자가 넣어준 입력이었던 </s><usr> 오늘도 좋은 하루!<sys>도 포함하여 반환되므로 챗봇의 답변만 확인하기 위해서는 <sys>를 기준으로 분할하여 뒷 부분만 가져와야 합니다.

In [18]:
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))


오늘도 좋은 하루네요.


만약 동일한 질문에도 KoGPT-2의 답변이 랜덤성을 갖기를 원한다면 do_sample=True, top_k=10 이라는 파라미터를 통해서 후보가 되는 단어 10개 중 랜덤으로 선택하여 생성하도록 유도할 수 있습니다.

In [19]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))


하루도 짝사랑처럼, 짝사랑처럼, 짝사랑처럼, 짝사랑은 짝사랑처럼, 짝사랑처럼, 짝사랑은 짝사랑처럼, 짝사랑처럼, 그리고 짝사랑은 짝사랑


지금까지의 과정을 함수로 만들어서 임의의 입력에 대해서 챗봇의 답변을 출력해봅시다.

In [20]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response


In [21]:
return_answer_by_chatbot('안녕! 반가워~')


'많이 반가운 일인가봐요.'

In [22]:
return_answer_by_chatbot('너는 누구야?')


'그분이 어떤 분이랑 관계하는 사람일 거예요.'